# Preparing the Enviorment


## Intsall Essential Packaeges

In [ ]:
!pip install super-image datasets transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.2 MB/s eta 0:00:00


## Imports

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
import os
from pathlib import Path
from typing import List, Union

from super_image.models import EdsrModel

## Paths and Directories

In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

# --- Verification ---
!ls /content/drive/MyDrive # Should list the contents of your "My Drive" folder

Mounted at /content/drive
'Colab Notebooks'   Education	        IFTTT   Neuromarketing-EHIA
 datasets	   'Google AI Studio'   manga


In [ ]:
ROOT_PATH = Path('/content/drive/MyDrive/datasets/sen2venus')

TACO_RAW_DIR = ROOT_PATH / 'TACO_raw_data'
os.makedirs(TACO_RAW_DIR, exist_ok=True)
print(f"Data will be saved to: {TACO_RAW_DIR}")

SELECTED_SUBSETS = [
    "SUDOUE-4",
    "SUDOUE-5",
    "SUDOUE-6"
]
TACO_FILE_PATHS = [TACO_RAW_DIR / f"{site_name}.taco" for site_name in SELECTED_SUBSETS]


NORMALIZED_SETS_DIR = ROOT_PATH / 'normalized_sets'
os.makedirs(NORMALIZED_SETS_DIR, exist_ok=True)
print(f"Normalaized datest will be saved to: {NORMALIZED_SETS_DIR}")

TRAIN_SAVE_DIR = NORMALIZED_SETS_DIR / 'train'
os.makedirs(TRAIN_SAVE_DIR, exist_ok=True)
print(f"Train data will be saved to: {TRAIN_SAVE_DIR}")

VAL_SAVE_DIR = NORMALIZED_SETS_DIR / 'val'
os.makedirs(VAL_SAVE_DIR, exist_ok=True)
print(f"Validation data will be saved to: {VAL_SAVE_DIR}")

TEST_SAVE_DIR = NORMALIZED_SETS_DIR / 'test'
os.makedirs(TEST_SAVE_DIR, exist_ok=True)
print(f"Test data will be saved to: {TEST_SAVE_DIR}")

Data will be saved to: /content/drive/MyDrive/datasets/sen2venus/TACO_raw_data
Normalaized datest will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets
Train data will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets/train
Validation data will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets/val
Test data will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets/test


# Step 1: Define PyTorch Datasets

In [ ]:
class PreNormalizedDataset(Dataset):
    """
    Efficiently reads pre-processed, sharded tensor files from disk.
    """
    def __init__(self, shard_dir: Union[str, Path]):
        self.shard_dir = Path(shard_dir)
        self.shard_paths: List[Path] = sorted(self.shard_dir.glob("*.pt"))

        if not self.shard_paths:
            raise ValueError(f"No shard files ('*.pt') found in {self.shard_dir}")

        # To calculate length, we check the size of the first shard and assume
        # all but the last are the same size.
        first_shard = torch.load(self.shard_paths[0])
        self.shard_size = len(first_shard)
        last_shard = torch.load(self.shard_paths[-1])
        self.length = (len(self.shard_paths) - 1) * self.shard_size + len(last_shard)

        # Simple cache to avoid re-loading the same shard consecutively
        self._cache = {}
        self._cached_shard_index = -1
        print(f"Initialized dataset from {self.shard_dir} with {self.length} samples across {len(self.shard_paths)} shards.")


    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        shard_index = idx // self.shard_size
        index_in_shard = idx % self.shard_size

        if shard_index != self._cached_shard_index:
            self._cache = torch.load(self.shard_paths[shard_index])
            self._cached_shard_index = shard_index

        # coupled with TACORGBDataset dataset class
        # each item in the shard is a squeezed dictionary with keys lr and hr
        squeezed_sample = self._cache[index_in_shard]
        return squeezed_sample['lr'], squeezed_sample['hr']

## Dataloader verification

In [ ]:
train_dataset = PreNormalizedDataset(TRAIN_SAVE_DIR)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Verify a batch
lr_batch, hr_batch = next(iter(train_loader))
print(f"Verification successful!")
print(f"LR batch shape: {lr_batch.shape}")
print(f"HR batch shape: {hr_batch.shape}")
print(f"LR batch dtype: {lr_batch.dtype}")
print(f"HR batch dtype: {hr_batch.dtype}")

Initialized dataset from /content/drive/MyDrive/datasets/sen2venus/normalized_sets/train with 4436 samples across 5 shards.
Verification successful!
LR batch shape: torch.Size([16, 3, 128, 128])
HR batch shape: torch.Size([16, 3, 256, 256])
LR batch dtype: torch.float32
HR batch dtype: torch.float32


In [ ]:
val_dataset = PreNormalizedDataset(VAL_SAVE_DIR)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
test_dataset = PreNormalizedDataset(TEST_SAVE_DIR)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Initialized dataset from /content/drive/MyDrive/datasets/sen2venus/normalized_sets/test with 556 samples across 1 shards.
Loaded 556 test samples.


# Step 2: Load the Pre-trained EDSR Model

## 2.1 Instantiate and Inspect the pre-trained EDSR model

In [ ]:
# The 'from_pretrained' method downloads the model configuration and weights.
# We must specify our desired scale factor. Based on your data shapes
# (LR: 128x128, HR: 256x256), our scale is 2.
scale = 2
model_id = 'eugenesiow/edsr-base'
model = EdsrModel.from_pretrained(model_id, scale=scale)

# Inspect the model architecture
# This is good practice to understand what we've loaded.
print("Model architecture loaded successfully:")
print(model)

config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

pytorch_model_2x.pt:   0%|          | 0.00/5.51M [00:00<?, ?B/s]

https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_2x.pt
Model architecture loaded successfully:
DataParallel(
  (module): EdsrModel(
    (sub_mean): MeanShift(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (add_mean): MeanShift(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (head): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (body): Sequential(
      (0): ResBlock(
        (body): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (1): ResBlock(
        (body): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (2): ResBlock(
        (body): Sequential(
       

## 2.2 Sanity Check: Pass one batch of data through the model

In [ ]:
# a crucial test to ensure the input/output dimensions are compatible.
print("\nPerforming a forward pass sanity check...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set to evaluation mode for this check

with torch.no_grad():
    # Get a single batch from our dataloader
    lr_batch, hr_batch = next(iter(train_loader))

    # Move the batch to the same device as the model
    lr_batch = lr_batch.to(device)

    # Perform a forward pass
    predictions = model(lr_batch)

    print(f"Sanity check successful!")
    print(f"Running on device: {device}")
    print(f"Model Input Shape (LR): {lr_batch.shape}")
    print(f"Model Output Shape (Predictions): {predictions.shape}")
    print(f"Target Shape (HR): {hr_batch.shape}")

# Compare output shape with the target High-Resolution shape
assert predictions.shape == hr_batch.shape, "Model output shape does not match target HR shape!"
print("Output shape matches target shape. Ready for training.")


Performing a forward pass sanity check...
Sanity check successful!
Running on device: cuda
Model Input Shape (LR): torch.Size([16, 3, 128, 128])
Model Output Shape (Predictions): torch.Size([16, 3, 256, 256])
Target Shape (HR): torch.Size([16, 3, 256, 256])
Output shape matches target shape. Ready for training.
